<img src="../images/alext_net.png" width = 600 height = 500>

【CONV1】： 96个11x11的卷积核，步长为4，没有零填充。参数：(11 * 11 * 3 +1) * 96<br>
【MAX POOL1】: chuangkoudaxiao 3x3, 步长为2。降低图像尺寸，轻微对抗目标便宜带来的影响。 输出尺寸：(55-3)/2 +1 = 27。参数个数：0<br>
【NORM1】:归一化层<br>
【CONV2】：96个5x5的卷积核，步长为1，使用零填充p=2. 输出特征图尺寸： 27x27x96<br>
【MAX POOL2】：256个5x5的卷积核，步长为1，使用零填充p=2。输出特征图：27x27x256<br>
【NORM2】<br>
【CONV3,CONV3】:384个3x3卷积核，步长为1，使用零填充p=1. 输出尺寸：13x13x384<br>
【CONV5】：256ge 3x3卷积核，步长为1，使用零填充p=1.<br>
【Max POOL3】：输出6x6x256<br>
【FC6,FC7,FC8】:全连接神经网络分类


In [1]:
import torch
import torchvision
import torchvision.models

from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

from PIL import ImageFile

### 制作图片数据索引train.txt和test.txt两个文件

In [2]:
import os

train_txt_path = os.path.join("catVSdog","train.txt")
train_dir = os.path.join("catVSdog","train_data")
valid_txt_path = os.path.join("catVSdog","test.txt")
valid_dir = os.path.join("catVSdog","test_data")

print(type(train_dir))


def gen_txt(txt_path, img_dir):
    f = open(txt_path, 'w')
 
    for root, s_dirs, _ in os.walk(img_dir, topdown=True):  # 获取 train文件下各文件夹名称
        for sub_dir in s_dirs:
            i_dir = os.path.join(root, sub_dir)             # 获取各类的文件夹 绝对路径
            img_list = os.listdir(i_dir)                    # 获取类别文件夹下所有png图片的路径
            for i in range(len(img_list)):
                if not img_list[i].endswith('jpg'):         # 若不是png文件，跳过
                    continue
                #label = (img_list[i].split('.')[0] == 'cat')? 0 : 1 
                label = img_list[i].split('.')[0]
                # 将字符类别转为整型类型表示
                if label == 'cat':
                    label = '0'
                else:
                    label = '1'
                img_path = os.path.join(i_dir, img_list[i])
                line = img_path + ' ' + label + '\n'
                f.write(line)
    f.close()

<class 'str'>


In [3]:
gen_txt(train_txt_path,train_dir)
gen_txt(valid_txt_path,valid_dir)

### 构建Dataset子类

In [4]:
from PIL import Image
from torch.utils.data import Dataset

class MyDataset(Dataset):
    #得到一个包含数据和标签的list
    def __init__(self,txt_path,transform = None, target_transform = None):
        fh = open(txt_path,'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0],int(words[1]))) #类别转换为整形int
            self.imgs = imgs
            self.transform = transform
            self.target_transform = target_transform
            
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB')
        if self.transform is not None:
            img =self.transform(img)
        return img,label
    
    def __len__(self):
        return len(self.imgs)

note:transform里边可以实现减均值、除标准差、随机裁剪、旋转、翻转、放射变换等操作

### 加载数据集和数据预处理
当Mydataset构建好，剩下的操作就交给DataLoder来加载数据集。在DataLoder中，会触发Mydataset中的getiterm函数读取一张图片的数据和标签，并拼接成一个batch返回，作为模型真正的输入。

In [5]:
pipeline_train= transforms.Compose([
    # 随机选转图片
    transforms.RandomHorizontalFlip(),
    # 将图片尺寸resize到227x227
    transforms.Resize((227,227)),
    # 将图片转化为tensor格式
    transforms.ToTensor(),
    # 正则化
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


pipeline_test = transforms.Compose([
    #将图片尺寸resize到227x227
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
])

train_data = MyDataset('./catVSdog/train.txt',transform = pipeline_train)
test_data = MyDataset('./catVSdog/test.txt',transform = pipeline_test)

#train_data 和test_data包含多有的训练与测试数据，调用DataLoader批量加载
trainloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=test_data, batch_size=32, shuffle=False)
# 类别信息也是需要我们给定的
classes = ('cat', 'dog') # 对应label=0，label=1

看一下最终制作的数据集图片和它们对应的标签：

In [ ]:
examples = enumerate(trainloader)
batch_idx, (example_data, example_label) = next(examples)
# 批量展示图片
for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.tight_layout()  #自动调整子图参数，使之填充整个图像区域
    img = example_data[i]
    img = img.numpy() # FloatTensor转为ndarray
    img = np.transpose(img, (1,2,0)) # 把channel那一维放到最后
    img = img * [0.5, 0.5, 0.5] + [0.5, 0.5, 0.5]
    plt.imshow(img)
    plt.title("label:{}".format(example_label[i]))
    plt.xticks([])
    plt.yticks([])
plt.show()